In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import numba
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../pax"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import waveform_utils
#from pax_utils import waveform_raw_utils
#from pax_utils import waveform_pe_utils

from pax.configuration import load_configuration

pp = pprint.PrettyPrinter(depth=2)



In [2]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(30, 15))

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    intr0  = None
    sum_wf = None
    event  = None

    df_zip_merged = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        ############################################################################################
        ############################################################################################

        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        #clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################

        jsonfilename = os.path.dirname(zipfilename) + '/pax_info.json'
        print("   CFG: '" + str(jsonfilename) + "'")
        
        cfg = event_utils.getConfig(jsonfilename)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

        
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        if (nInteractions != 1): continue
        
        
        ############################################################################################
        ############################################################################################

        intr0 = None
        
        if (len(interactions) > 0):
            
            intr0 = interactions[0]
            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
              
        ############################################################################################
        # S2 Window from Event
        ############################################################################################
        
        left  = event.main_s2.left
        right = event.main_s2.right
        
    
        print()
        
        
        ############################################################################################
        # Raw Waveforms
        ############################################################################################
    
        arr_summed_waveform_raw_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_raw')[left:right]
        df_channels_waveforms_raw   = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'tpc', True)
        arr_summed_waveform_raw_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_raw, event)[left:right]
        
        #f = plt.figure(figsize=(10, 5))
        #plt.plot(arr_summed_waveform_raw_evt, color='b', label='Raw Summed Waveform from Event') 
        #plt.plot(arr_summed_waveform_raw_df , color='r', label='Raw Summed Waveform by Hand DF') 
        #plt.legend(fontsize=18)
        
        eq_raw = waveform_utils.compareWaveforms(arr_summed_waveform_raw_evt, arr_summed_waveform_raw_df)
                                                                                      
        
        ############################################################################################
        # TPC Summed Waveform
        ############################################################################################
        
        arr_summed_waveform_tpc_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc')[left:right]
        df_channels_waveforms_tpc   = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'tpc', False)
        arr_summed_waveform_tpc_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_tpc, event)[left:right]
        
        #f = plt.figure(figsize=(10, 5))
        #plt.plot(arr_summed_waveform_tpc_evt, color='b', label='TPC Summed Waveform from Event') 
        #plt.plot(arr_summed_waveform_tpc_df , color='r', label='TPC Summed Waveform by Hand DF') 
        #plt.legend(fontsize=18)
        
        eq_tpc = waveform_utils.compareWaveforms(arr_summed_waveform_tpc_evt, arr_summed_waveform_tpc_df)
                                                                                      
       
          
        ############################################################################################
        # Top Summed Waveform
        ############################################################################################
        
        arr_summed_waveform_top_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_top')[left:right]
        df_channels_waveforms_top   = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'top', False)
        arr_summed_waveform_top_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top, event)[left:right]
        
        #f = plt.figure(figsize=(10, 5))
        #plt.plot(arr_summed_waveform_top_evt, color='b', label='TOP Summed Waveform from Event') 
        #plt.plot(arr_summed_waveform_top_df , color='r', label='TOP Summed Waveform by Hand DF') 
        #plt.legend(fontsize=18)
        
        eq_t = waveform_utils.compareWaveforms(arr_summed_waveform_top_evt, arr_summed_waveform_top_df)
        
    
        display(df_channels_waveforms_top[0:5][:])
        
                
        #############################################################################################
        ## Sanity
        #############################################################################################
#
        #event_sum_wf  = np.sum(evt_summed_waveform_top)
        #event_sum_s2s = np.sum(df_pkl_s2s.drop(columns=['event_number']).as_matrix(), axis=1) 
        #
        #print()
        #print("Event Sum Waveform: " + str(event_sum_wf))
        #print("Event Sum S2's:     " + str(event_sum_s2s))

       
    
        #############################################################################################
        #############################################################################################

        assert(18 == len(event.get_sum_waveform_names()))
        
            
        ############################################################################################
        ############################################################################################
    
        continue
            
    
    ################################################################################################
    ################################################################################################
    
    #plt.legend()
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged, event, intr0, sum_wf

    

In [3]:

####################################################################################################
####################################################################################################

zipfilename    = '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
nEventsPerFile = 2


df_zip_merged, event, intr, wf_sum = processPklEvents(zipfilename)


    

File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
   event_number: 0
   CFG: '../pax_run/pax_output/Mar25/instructions_000000/pax_info.json'
File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
   event_number: 1
   CFG: '../pax_run/pax_output/Mar25/instructions_000000/pax_info.json'

Arrays are equal within 1e-05
Arrays are equal within 1e-05
Arrays are equal within 1e-05


,channel,left,right,length,sum,raw_data
0,0,105424,105791,368,1.715228,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,65458,148393,82936,2.263461,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,5,10394,127133,116740,1.034101,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,9,105400,105501,102,1.293108,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
10,10,105580,105681,102,1.385255,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
15,15,13184,125603,112420,2.072034,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
16,16,105486,105789,304,4.501663,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17,17,105562,105661,100,1.349055,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
18,18,105482,105583,102,0.958684,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19,19,105410,105639,230,2.223428,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
